# MSDM5002 final project demo 
Hello, Here is the demo game.

### If you have any uninstall requirements, just use conda or pip to install it, and run the code below
For example, the 'ctypes' is missing, run conda install ctypes

In [6]:
# -*- coding: utf-8 -*-
# @Time    : 27/11/2021 4:06 pm
# @Author  : ZIFENG NEIL CHEN
# @FileName: game.py.py
# @Software: PyCharm
# @Github    ：https://github.com/NNNeil-C/
import game_gui
import pygame
import game_utils
from models import piece, player
import numpy as np
import sys
import platform

def make_drop(board, current_player, row, col, Alice, ui):
    # drop on board
    board[row, col] = 1 if current_player is Alice else -1
    print(board)
    # update ui
    current_type = piece.piece.black_piece if current_player is Alice else piece.piece.white_piece
    current_piece = piece.piece(current_type)
    ui.drop_piece(current_piece, col, row)


def do_something_after_wins(win_player, Alice):
    if win_player is Alice:
        print("Black wins")
    else:
        print("White wins")


def ai_plays(player, game_board, Alice, last_x, last_y, last_piece):
    pygame.event.set_blocked(pygame.MOUSEBUTTONDOWN)
    x, y = game_utils.ask_monte_carlo_search_tree(game_board, last_x, last_y, last_piece)
    make_drop(game_board, player, x, y, Alice, ui)
    # check if the last drop wins
    is_win = False
    if game_utils.check_win_cpp(game_board, x, y):
        print("someone wins")
        do_something_after_wins(player, Alice)
        is_win = True
    pygame.event.clear()
    pygame.event.set_allowed(pygame.MOUSEBUTTONDOWN)
    return is_win

### set Alice_is_ai = False if you want to play first   
### set Alice_is_ai = True if you want to play second   
### checkout the output for game log   

In [ ]:
# ai first or human first
# set Alice_is_ai = False if you want to play first
Alice_is_ai = False
# set Alice_is_ai = True if you want to play second
#Alice_is_ai = True

ui = game_gui.game_ui()
clock = pygame.time.Clock()
Alice = player.player(piece.piece.black_piece, is_ai=Alice_is_ai)
Bob = player.player(piece.piece.white_piece, is_ai=not Alice_is_ai)
current_player = Alice
so_file_path = r"./mcst/build/lib.macosx-11.0-arm64-3.8/mcst_helper.cpython-38-darwin.so"
if 'Windows' in platform.platform():
    so_file_path = r"./mcst/build/lib.Windows*/mcst_helper.dll"
game_utils.load_dynamic_lib(so_file_path)

game_board = np.zeros((game_utils.game_board_size, game_utils.game_board_size), dtype=np.int64)

game_is_over = False
while True:
    clock.tick(60)
    if current_player.is_ai:
        ai_plays(current_player, game_board, Alice, -1, -1, 0)
        current_player = Alice if current_player is Bob else Bob
    for event in pygame.event.get():
        # quit game
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()
        # click down
        if event.type == pygame.MOUSEBUTTONDOWN:
            if game_is_over:
                continue
            if not current_player.is_ai:
                # get position
                row, col = game_utils.get_click_position(event)
                # check position
                if game_utils.is_valid_position(game_board, col, row):
                    make_drop(game_board, current_player, row, col, Alice, ui)
                    # check if the last drop wins
                    if game_utils.check_win_cpp(game_board, row, col):
                        print("someone wins")
                        do_something_after_wins(current_player, Alice)
                        game_is_over = True
                    # player switch
                    current_player = Alice if current_player is Bob else Bob
                    if current_player.is_ai:
                        is_win = ai_plays(current_player, game_board, Alice, row, col, game_board[row][col])
                        game_is_over |= is_win
                        current_player = Alice if current_player is Bob else Bob

check visit time
0 0 267 525 300 104 239 0 
0 276 348 56 198 1285 310 514 
0 216 178 521 0 315 1222 413 
0 282 125 0 0 0 269 109 
0 889 4316 110 0 589 100 298 
0 577 458 107164 110 411 61 298 
0 0 1603 850 213 49 238 0 
0 0 0 0 0 0 0 0 
hello
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[0, 0, 0]
start deduction
root->last_piece is 1deduction decisive_move test start
current piece is -1
d_move.first is -1search_counter : 107330
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
[255, 255, 255]
